In [1]:
import pandas as pd
import numpy as np

from scipy.special import betaln
from collections import Counter
from matplotlib import pyplot as plt
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/validation_data.csv')

try:
    total = pd.read_csv('processed_data/train.csv')
except FileNotFoundError:
    total = []
    for d in df.to_dict('records'):
        s1, s2 = d['less_toxic'], d['more_toxic']
        if s1 < s2:
            total.append([s1, s2, 1])
        else:
            total.append([s2, s1, 0])

    temp = pd.DataFrame(total, columns=['s1', 's2', 'toxic'])

    total = []

    for g in temp.groupby(['s1', 's2']):
        pair, frame = g
        second = frame['toxic'].sum()
        first = len(frame) - second
        total.append({'s1': pair[0], 's2': pair[1], 'first': first, 'second': second})

    total = pd.DataFrame(total)
    total.to_csv('processed_data/train.csv', index=False)
total


,first,s1,s2,second,beta,odds,log_odds
0,2,\n\nThe comment directly above this one are fr...,"""::you wouldn't know Makoto from Street Fighte...",1,0.242857,0.320755,-1.137079e+00
1,3,\n\nwhy should people have to read crap posted...,I just read what you wrote; neither of the tw...,0,0.004995,0.005020,-5.294309e+00
2,3,\n\nwhy should people have to read crap posted...,"""\n\nI don't know which """"personal attacks"""" y...",0,0.004995,0.005020,-5.294309e+00
3,2,\n\nwhy should people have to read crap posted...,"Yes it is. This is NOT an encyclopedia, it is ...",1,0.028638,0.029482,-3.523973e+00
4,2,\nGo F yourself you cottonheadednittymuggins.,people like you are the reason wikipedia is us...,1,0.348485,0.534884,-6.257059e-01
5,0,"\nU POUR ADMIN, U UPDATE VANDAL COUNT, WHILE R...",Your relentless edit warring \n\nYour message...,3,0.996970,329.000000,5.796058e+00
6,1,\n\n koreans claim \n\ni see youve seen the ...,YOU'RE IN BIG TROUBLE\n\nI'VE ALREADY CALLED ...,2,0.856935,5.989817,1.790061e+00
7,1,\n\n koreans claim \n\ni see youve seen the ...,"""\n\nWell, it appears that that vandal has res...",2,0.484163,0.938596,-6.336961e-02
8,2,\n\nFailure to follow the guidelines of WP:NO...,I'd rather he took the time to report me than...,1,0.703963,2.377953,8.662399e-01
9,0,\n\nFailure to follow the guidelines of WP:NO...,Against community and you don't know what devi...,3,0.782188,3.591124,1.278465e+00


In [3]:
def beta_difference(a1, b1, a2, b2):
    total = 0
    if a2 > a1:
        return 1 - beta_difference(a2, b2, a1, b1)
    
    for i in range(0, a2):
        term = betaln(a1+i, b1+b2) - np.log(b2+i) - betaln(1+i, b2) - betaln(a1, b1)
        total += np.exp(term)
    return total

def beta_means(a1, b1, a2, b2):
    return (a1 * b2) / (a1 * b2 + a2 * b1)

In [4]:
pos_counts = Counter(df['less_toxic'].to_list())
neg_counts = Counter(df['more_toxic'].to_list())

all_sentences = set(pos_counts) | set(neg_counts)

for s in all_sentences:
    if s not in pos_counts:
        pos_counts[s] = 0
    if s not in neg_counts:
        neg_counts[s] = 0

tox_prob = []
adjusted = []

# for s1, s2 in tqdm(df[['less_toxic', 'more_toxic']].to_numpy()):
#     neg1, pos1 = neg_counts[s1], pos_counts[s1]
#     neg2, pos2 = neg_counts[s2], pos_counts[s2]

#     difference = beta_difference(neg_counts[s1] + 1, pos_counts[s1] + 1, neg_counts[s2] + 1, pos_counts[s2] + 1)
#     tox_prob.append(difference)

pos_1 = []
neg_1 = []
pos_2 = []
neg_2 = []

for s1, s2 in tqdm(total[['s1', 's2']].to_numpy()):
    a1, b1 = neg_counts[s1], pos_counts[s1]
    a2, b2 = neg_counts[s2], pos_counts[s2]
    
    difference = beta_difference(a1 + 1, b1 + 1, a2 + 1, b2 + 1)
    adjusted.append(difference)
    pos_1.append(b1)
    pos_2.append(b2)
    neg_1.append(a1)
    neg_2.append(a2)

total['beta'] = adjusted
total['odds'] = total['beta'] / (1-total['beta'])
total['log_odds'] = np.log(total['odds'])



100%|██████████| 10108/10108 [00:00<00:00, 18315.61it/s]


In [5]:
total.sort_values(by='odds')[::-1]

,first,s1,s2,second,beta,odds,log_odds
7439,0,""".\n\n""""This emphasis also led Boas to conclud...",YOU'RE A FUCKING LIAR WHO DOESN'T KNOW HOW TO ...,3,0.999999,1.144065e+06,13.950098
2566,0,Petition for Wikipedia to Unban Virgin Media ...,Wow...! \n\nYou're pretty lame. Too lazy to g...,3,0.999995,1.847550e+05,12.126786
4498,0,this 6 mil.. is the biggest lie in history !!,Fuck you people are dumb.... Facebook IS a pub...,3,0.999995,1.847550e+05,12.126786
3540,0,"Wikipedia is full of shit, don't waste your t...",YOU SUCK \n\nI HATE YOU> YOU CAN FUCK MY ASS ...,3,0.999967,3.064362e+04,10.330180
9957,0,Wikipedia is a repetition of the old joke that...,so you assholes are high on power like george ...,3,0.999949,1.944700e+04,9.875448
9275,0,It is also getting old when I've got a guy lik...,WHO ARE YOU FUCKING CUNT HEADS TO TELL ME THAT...,3,0.999949,1.944700e+04,9.875448
1755,0,I gave the source. Clearly you did not read i...,YOU ARE A FUCKING NERD. GO KILL YOURSELF YOU N...,3,0.999949,1.944700e+04,9.875448
645,0,Bad accent \n\nHer attempts to do a working-c...,J.delanoy IS A NIGGER ==== J.delanoy IS A N...,3,0.999949,1.944700e+04,9.875448
5917,0,"""\n\nGwen, maybe I am missing something.... t...",WHO ARE YOU FUCKING CUNT HEADS TO TELL ME THAT...,3,0.999949,1.944700e+04,9.875448
3130,0,The sooner you people just cut the crap and g...,warning? \n\nwhy did you warn me? for what? i...,3,0.999949,1.944700e+04,9.875448


In [6]:
from transformers import BertTokenizer
from models.bert_ranking import BertForRanking
from models.dataset import RankingDataset


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', model_max_length=128)
model = BertForRanking.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForRanking: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForRanking from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForRanking from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForRanking were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['encoder.laye

In [9]:
s = tokenizer(total['s1'][:5].to_list(), return_tensors='pt', padding=True, truncation=True)
s2 = tokenizer(total['s2'][:5].to_list(), return_tensors='pt', padding=True, truncation=True)

dataset = RankingDataset(s, s2, total['beta'][:5].to_list())

In [10]:
r = model(**dataset[:5])

/anaconda3/lib/python3.7/site-packages/torch/nn/functional.py:2398: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn("reduction: 'mean' divides the total loss by both the batch size and the support size."


In [11]:
r

(tensor(0.2491, grad_fn=<KlDivBackward>), tensor([[ 0.2096,  0.3119],
         [-0.0784,  0.1681],
         [-0.0784,  0.1926],
         [-0.0784,  0.1250],
         [-0.0328,  0.0732]], grad_fn=<CatBackward>))